In [ ]:
from typing import List
import folium
import numpy as np
import pandas as pd

In [ ]:
class Point:
    def __init__(self, lat, long):

        self.lat = lat
        self.long = long

class Polygon:
    def __init__(self, point:List):
        """
        points: a list of Points in clockwise order.
        """
        self.points = point

    @property
    def edges(self):
        ''' Returns a list of tuples that each contain 2 points of an edge '''
        edge_list = []
        for i,p in enumerate(self.points):
            p1 = p
            p2 = self.points[(i+1) % len(self.points)]
            edge_list.append((p1,p2))

        return edge_list

    @property
    def bounds(self):
        min_lat = min(point.lat for point in self.points)
        max_lat = max(point.lat for point in self.points)
        min_long = min(point.long for point in self.points)
        max_long = max(point.long for point in self.points)
        
        return (min_lat, min_long, max_lat, max_long)

    def contains(self, point):
        '''
        Check for x and y conditions.
        y: the point should be between the y axis of both the edges. ay<py<by
    
        x: the point should lie on the left of the line or 
            when you put the point on the equation of line: yp-y1 = [(y2-y1)/(x2-x1)]*[xp-x1]
                                                            [(yp-y1)*(x2-x1)]/(y2-y1)=(xp-x1)
            the xp should be less than (left) the line:  [(yp-y1)*(x2-x1)]/(y2-y1) + x1 > xp 
        '''
        count = 0
        for edge in self.edges:
            # print(edge)
            A,B = edge[0], edge[1]
            # make sure A is lower point of the edge
            if A.long >B.long:
                A,B = B,A

            # check for y condition
            if A.long<=point.long<B.long:
                # check for x condition
                if  (A.lat + ((point.long - A.long) * (B.lat - A.lat) / (B.long - A.long)) ) > point.lat:
                    count+=1

        return not count%2==0 # even is outside


In [ ]:
# Define the Southland boundary as a polygon
southland_boundary_coords = [[-118.3324111792,33.9991332253],[-118.334551407,34.1154302087],
                            [-118.1283374082,34.1180331638],[-118.1261971804,34.0017397543],
                            [-118.3324111792,33.9991332253]]
southland_boundary_coords = [Point(coords[1], coords[0]) for coords in southland_boundary_coords]

southland_polygon = Polygon(southland_boundary_coords)
anchor1 = Point(  34.090, -118.270)  # Adjusted to fall within the silver lake
anchor2 = Point(  34.003, -118.230,) # Adjusted to fall within vernon
buffer_radius = 50  # ~5 km
decay_factor = 0.2   # Controls decay distance
scale_factor = 1.0 # controls comfort zone
# random sampling weights
w1, w2 = 0.7, 0.3  # biased weights
num_samples = 13      # Number of crimes to generate

In [ ]:
def sample_distance_decay(anchor, shape, scale):
    '''
    Samples a crime location around an anchor point using distance decay based on a gamma distribution.

    Parameters:
    - anchor (Point): The anchor point (home or work location).
    - shape (float): The shape parameter (k) for the gamma distribution. Higher values shift the peak further out, controlling the spread of crime..
    - scale (float): The scale parameter (theta) for the gamma distribution. Smaller values create a sharper decay, concentrating crimes closer to the anchor.

    Returns:
    - Point: A new Point object representing the latitude and longitude of the sampled crime location.
    '''
    # Sample distance from a gamma distribution with specified shape and scale
    distance = np.random.gamma(shape, scale)
    # Generate a random angle to simulate direction
    angle = np.random.uniform(0, 2 * np.pi)
    # Convert polar coordinates (distance and angle) to Cartesian coordinates
    dx = distance * np.cos(angle)
    dy = distance * np.sin(angle)
    # Calculate new coordinates by applying the offset to the anchor's location
    lat = anchor.lat + dx
    lon = anchor.long + dy
    return Point(lat, lon)

def haversine_distance(point1, point2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = np.radians(point1)
    lat2, lon2 = np.radians(point2)
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Earth's radius (mean radius = 6,371 km)
    R = 6371.0
    distance = R * c  # in kilometers
    return distance

def is_valid_point(point, anchor1, anchor2, buffer_radius, polygon):
    distance_from_anchor1 = haversine_distance((point.lat, point.long), (anchor1.lat, anchor1.long))
    distance_from_anchor2 = haversine_distance((point.lat, point.long), (anchor2.lat, anchor2.long))
    if polygon.contains(point):
        if (distance_from_anchor1 > buffer_radius) and (distance_from_anchor2 > buffer_radius):
            return True
    return False

def generate_samples(num_samples, anchors,anchor_prob,  decay_factor,scale_factor, polygon):
    samples = []
    anchor_list = np.random.choice(a=anchors, p=anchor_prob, size=num_samples)
    np.random.shuffle(anchor_list)
    while len(samples) < num_samples:
    # obtain lat, long after sampling distance 
        anchor = anchor_list[len(samples)]
        point = sample_distance_decay(anchor,  decay_factor,scale_factor)
        # Check if point is valid
        if is_valid_point(point, anchors[0], anchors[1], buffer_radius, polygon):
            samples.append([point.lat, point.long]) 
            print(f"sample {len(samples)}/{num_samples} generated")
    return samples


In [ ]:

samples = generate_samples(13,[anchor1,anchor2],[w1,w2],  decay_factor, scale_factor,southland_polygon)


In [ ]:

def visualise(samples):
    # Convert to DataFrame for analysis
    crime_data = pd.DataFrame(samples, columns=["Latitude", "Longitude"])
    
    # Plot crime data on a map
    map_center = [34.0, -118.0]  # Center of Southland area

    # Initialize map centered on Southland
    m = folium.Map(location=map_center, zoom_start=17)

    # Plot each crime location
    for idx, row in crime_data.iterrows():
        folium.CircleMarker(
            location=(row['Latitude'], row['Longitude']),
            radius=3,
            color='red',
            fill=True,
            fill_color='red'
        ).add_to(m)

    # Add markers for anchor points
    folium.Marker((anchor1.long,anchor1.lat), tooltip="Anchor 1", icon=folium.Icon(color="blue")).add_to(m)
    folium.Marker((anchor2.long,anchor2.lat), tooltip="Anchor 2", icon=folium.Icon(color="green")).add_to(m)

    # Display map
    m.save("southland_crime_map.html")
    return m
